In [18]:
# INSTALANDO AS BIBLIOTECAS
%pip install polars
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
## importando bibliotecas necessarias
import polars as pl
import numpy as np

In [72]:
# IMPORTANDO O ARQUIVO QUE SERA UTILIZADO NO PIPELINE
# NO ETL ESSA ETAPA E O "E" DE EXTRAÇÃO
coffee = pl.read_csv('data/coffee.csv', encoding='UTF-8')

# INICIANDO A ANALISE EXPLORATORIA (BÁSICA) DOS DADOS
coffee.head()

Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
str,str,str,f64,f64,str,bool,bool,bool
"""C""","""Dim Kavu""","""Others""",4.6,206.0,"""$$""",false,null,null
"""C""","""Коферум""","""Cafe""",5.0,24.0,"""$$""",false,null,true
"""C""","""Кофейня Світ Ч...","""Coffee shop""",5.0,11.0,"""$$""",false,null,true
"""C""","""Кофейня Starco...","""Coffee shop""",4.4,331.0,"""$$""",false,true,true
"""C""","""Кофейня ""Frien...","""Coffee shop""",5.0,12.0,"""$$""",false,true,true


In [73]:
coffee.shape

(200, 9)

In [74]:
coffee.describe()

describe,Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
str,str,str,str,f64,f64,str,f64,f64,f64
"""count""","""200""","""200""","""200""",200.0,200.0,"""200""",200.0,200.0,200.0
"""null_count""","""0""","""0""","""0""",2.0,2.0,"""0""",0.0,60.0,56.0
"""mean""",null,null,null,4.663636,622.494949,null,0.175,1.0,1.0
"""std""",null,null,null,0.221648,1400.898666,null,0.380921,0.0,0.0
"""min""","""A""","""""®КАВА БАЗА®""""","""Cafe""",3.9,3.0,"""$""",0.0,0.0,0.0
"""max""","""J""","""Чай Кофе""","""Others""",5.0,17937.0,"""$$$""",1.0,1.0,1.0
"""median""",null,null,null,4.7,271.5,null,0.0,1.0,1.0


In [75]:
for col in coffee.get_columns():
    print(f'{col.name} - {col.is_null().sum()}')

Region - 0
Place name - 0
Place type - 0
Rating - 2
Reviews - 2
Price - 0
Delivery option - 0
Dine in option - 60
Takeout option - 56


In [76]:
# PROCESSO DE TRANSFORMAÇÃO
# TROCANDO OS NA POR FALSE NAS COLUNAS BOOLEANAS
# Como nesssas colunas não existem False subentende-se que os 'nan' são False (COMO PEDE O EX)
coffeeDF = coffee #criando um novo dataframe para deixar o programa escalável

for column in ['Dine in option', 'Takeout option', 'Delivery option']:
    coffeeDF = coffeeDF.with_columns(column, coffeeDF[column].fill_null(False))

coffeeDF

Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
str,str,str,f64,f64,str,bool,bool,bool
"""C""","""Dim Kavu""","""Others""",4.6,206.0,"""$$""",false,false,false
"""C""","""Коферум""","""Cafe""",5.0,24.0,"""$$""",false,false,true
"""C""","""Кофейня Світ Ч...","""Coffee shop""",5.0,11.0,"""$$""",false,false,true
"""C""","""Кофейня Starco...","""Coffee shop""",4.4,331.0,"""$$""",false,true,true
"""C""","""Кофейня ""Frien...","""Coffee shop""",5.0,12.0,"""$$""",false,true,true
"""C""","""Racers Coffee ...","""Espresso bar""",4.6,367.0,"""$$""",false,true,true
"""C""","""Займемся Кофе""","""Coffee shop""",4.6,200.0,"""$$""",false,true,true
"""C""","""Кофейня Rit Ri...","""Coffee shop""",4.6,292.0,"""$$""",false,false,true
"""C""","""Кав'ярня My co...","""Coffee shop""",4.8,31.0,"""$$""",false,true,true


In [77]:
# SUBSTITUNDO OS 'nan' DAS COLUNAS COM VARIAVEIS NÚMERICAS PELA MÉDIA
coffeeDF = coffeeDF.with_columns('Rating', coffeeDF['Rating'].fill_null(coffeeDF['Rating'].mean()))

In [78]:
coffeeDF

Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
str,str,str,f64,f64,str,bool,bool,bool
"""C""","""Dim Kavu""","""Others""",4.6,206.0,"""$$""",false,false,false
"""C""","""Коферум""","""Cafe""",5.0,24.0,"""$$""",false,false,true
"""C""","""Кофейня Світ Ч...","""Coffee shop""",5.0,11.0,"""$$""",false,false,true
"""C""","""Кофейня Starco...","""Coffee shop""",4.4,331.0,"""$$""",false,true,true
"""C""","""Кофейня ""Frien...","""Coffee shop""",5.0,12.0,"""$$""",false,true,true
"""C""","""Racers Coffee ...","""Espresso bar""",4.6,367.0,"""$$""",false,true,true
"""C""","""Займемся Кофе""","""Coffee shop""",4.6,200.0,"""$$""",false,true,true
"""C""","""Кофейня Rit Ri...","""Coffee shop""",4.6,292.0,"""$$""",false,false,true
"""C""","""Кав'ярня My co...","""Coffee shop""",4.8,31.0,"""$$""",false,true,true


In [79]:
# REMOVENDO AS LINHAS ONDE AS REVIEWS SÃO NULAS COMO PEDE O EX
coffeeDF = coffeeDF.with_columns('Reviews', coffeeDF['Reviews'].is_not_null())

In [80]:
# VERIFICANDO SE O FILL MUDOU MUITO AS ESTATISTICAS
coffeeDF.head()

Region,Place name,Place type,Rating,Reviews,Price,Delivery option,Dine in option,Takeout option
str,str,str,f64,bool,str,bool,bool,bool
"""C""","""Dim Kavu""","""Others""",4.6,true,"""$$""",false,false,false
"""C""","""Коферум""","""Cafe""",5.0,true,"""$$""",false,false,true
"""C""","""Кофейня Світ Ч...","""Coffee shop""",5.0,true,"""$$""",false,false,true
"""C""","""Кофейня Starco...","""Coffee shop""",4.4,true,"""$$""",false,true,true
"""C""","""Кофейня ""Frien...","""Coffee shop""",5.0,true,"""$$""",false,true,true


In [81]:
for col in coffeeDF.get_columns():
    print(f'{col.name} - {col.is_null().sum()}')

Region - 0
Place name - 0
Place type - 0
Rating - 0
Reviews - 0
Price - 0
Delivery option - 0
Dine in option - 0
Takeout option - 0


In [87]:
# PARTE DO LOAD
# Como não iremos utilizar um BD para acessar os arquivos vou apenas salvar a file em .csv
coffeeDF.write_csv('data/coffe_etl.csv')

In [99]:
# FIM DO ETL
# PROXIMO PASSO - ANALISE